In [ ]:
#@title <b><font color="red" size="+3">↓</font><font color="black" size="+3"> Configuració 🏗️</font></b> 

!python -m pip install paddlepaddle-gpu==2.3.2 -f https://mirror.baidu.com/pypi/simple
!git clone https://github.com/PaddlePaddle/PaddleGAN
%cd /content/PaddleGAN
!python setup.py develop
!pip install -r requirements.txt
!python -m pip install visualdl -i https://mirror.baidu.com/pypi/simple

!sudo apt install imagemagick

from IPython.display import clear_output
clear_output()

In [ ]:
#@title <b><font color="red" size="+3">↓</font><font color="black" size="+3"> Pujar el vídeo de la locució 🧑📹</font></b> 

#@markdown **Podeu fer el crop del vídeo aquí: https://ezgif.com/crop-video**

#@markdown **Podeu convertir el vídeo aquí: https://convert-video-online.com**

%cd /content/PaddleGAN/

!rm -rf video
!mkdir video

import os
from google.colab import files
from IPython.display import clear_output


uploaded = list(files.upload().keys())

if len(uploaded) > 1:
  raise ValueError('You cannot upload more than one video at a time!')

vid = uploaded[0]
os.rename(vid, vid.replace(" ", ""))
vid = vid.replace(" ", "")
!mv -f $vid video/driving.mp4
vid = 'video/driving.mp4'

clear_output()

In [ ]:
#@title <b><font color="red" size="+3">↓</font><font color="black" size="+3"> Pujar el retrat ✏😐</font></b> 

%cd /content/PaddleGAN/

!rm -rf raw_images
!mkdir raw_images
uploaded = files.upload()

i = 0
raw_photolist = []
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  pho = "photo-" + str(i) + "." + fn.split(".")[1]
  !mv -f $fn raw_images/$pho
  !mogrify -resize 512x512! raw_images/$pho
  raw_photolist.append(pho)
  i += 1

clear_output()

In [ ]:
#@title <b><font color="red" size="+3">↓</font><font color="black" size="+3"> Genera el deepfake ⚙</font></b> 

%cd /content/PaddleGAN/applications/

for PHOTO in raw_photolist:
  !python -u tools/first-order-demo.py  \
     --driving_video '../video/driving.mp4' \
     --source_image '../raw_images/{PHOTO}' \
     --ratio 0.4 \
     --relative \
     --adapt_scale \
     --image_size 512 \
     --filename '{PHOTO}.mp4'

In [ ]:
#@title <b><font color="red" size="+3">↓</font><font color="black" size="+3"> Mostra el deepfake 📺</font></b> 


import io
import base64
from IPython.display import HTML

play_html = ''
for PHOTO in raw_photolist:
    video = io.open('/content/PaddleGAN/applications/output/%s.mp4' % PHOTO, 'r+b').read()
    encoded = base64.b64encode(video)
    play_html = play_html + ('<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /> </video>'.format(encoded.decode('ascii')))

clear_output()
HTML(data=play_html)

In [ ]:
#@title <b><font color="red" size="+3">↓</font><font color="black" size="+3"> Descarrega el resultat ⏬</font></b> 

!zip -FSr /content/PaddleGAN/applications/output.zip /content/PaddleGAN/applications/output
files.download('/content/PaddleGAN/applications/output.zip')